In [4]:
import datetime as dt
import wget
import os
import pandas as pd
from zipfile import ZipFile

In [5]:
# só extrai o arquivo dado como parâmetro para a pasta ./Dados/temp
def extrai_arquivo(arq):
    try:
        ZipFile(arq, 'r').extractall('./Dados/temp/')
    except:
        print(f"erro ao extrair {arq}")

In [6]:
# baixa dados, concatena em um dataframe só e salva em um .csv
# limpa depois
def baixa_e_concatena(ticker, timeframe, ano_inicial):
    ano_corrente, mes_corrente, dia_corrente = [dt.date.today().year, dt.date.today().month, dt.date.today().day]
    
    # baixa dados da binance conforme ticker e timeframe selecionados para a pasta ./Dados/
    # timeframes disponiveis: 12h 15m 1d 1h 1m 1mo 1s 1w 2h 30m 3d 3m 4h 5m 6h 8h
    # tickers disponiveis: https://data.binance.vision/?prefix=data/spot/monthly/klines/
    url = "https://data.binance.vision/data/spot/monthly/klines/"
    if not os.path.exists(f"./Dados/Processados/{ticker}-{timeframe}.csv"):
        for ano in range(ano_inicial, ano_corrente+1):
            for mes in range(1,12+1):
                mes = str(mes).zfill(2)
                if not ((os.path.exists(f"./Dados/temp/{ticker}-{timeframe}-{ano}-{mes}.zip"))):
                    try:
                        wget.download(f"{url}{ticker}/{timeframe}/{ticker}-{timeframe}-{ano}-{mes}.zip"
                                      , out = f"./Dados/temp/")
                        pass
                    except:
                        print(f"\nFalha ao baixar {url}{ticker}/{timeframe}/{ticker}-{timeframe}-{ano}-{mes}.zip")
                else:
                    print(f"{ano}/{mes} já baixado")
    else:
        print(f"{ticker}-{timeframe} já processado")
        return
    
    # cria uma lista de arquivos do ticker e timeframe selecionado
    lista_arquivos = os.listdir("./Dados/temp/")
    lista_arquivos = [x for x in lista_arquivos if x.startswith(f"{ticker}-{timeframe}")]
    lista_arquivos[-5:]
    
    # cria um dataframe vazio pra colocar todos os dados dentro
    nomes = ["Open time","Open","High","Low","Close","Volume","Close time","Quote asset volume"
                                 ,"Number of trades","Taker buy base asset volume","Taker buy quote asset volume","Ignore"]
    df = pd.DataFrame(columns = nomes)
    
    # concatena tudo em um CSV e deixa na pasta ./Dados/Processados/
    for arq in lista_arquivos:
        extrai_arquivo(f"./Dados/temp/{arq}")
        df = pd.concat([df, pd.read_csv(f'./Dados/temp/{arq[:-4]}.csv', sep=',',decimal='.'
                                   , encoding='latin1', names=nomes, header=None)], ignore_index=True, copy=False)
        os.remove(f"./Dados/temp/{arq[:-4]}.csv")
    df.drop("Ignore", inplace=True, axis=1)
    df.set_index("Open time", inplace=True)
    df.to_csv(f"./Dados/Processados/{ticker}-{timeframe}.csv")
    
    print(f"./Dados/Processados/{ticker}-{timeframe}.csv")
    
    # deleta tudo que é temporario e já foi processado
    for arq in lista_arquivos:
        os.remove(f"./Dados/temp/{arq}")
    
    return

In [7]:
# o de 1s demora muito. Pesa 16GB o arquivo final enquanto o de 1m pesa 300MB. tem que ver se vale a pena mesmo.
ticker = "BTCUSDT"
timeframes = ["1m", "5m", "15m", "30m", "1h", "2h", "4h", "8h", "1d"] # removi o "1s" pq o csv somente já pesa 16GB. inviável
for timeframe in timeframes:
    baixa_e_concatena(ticker= ticker, timeframe=timeframe, ano_inicial=2017)

BTCUSDT-1m já processado
BTCUSDT-5m já processado
BTCUSDT-15m já processado
BTCUSDT-30m já processado
BTCUSDT-1h já processado
BTCUSDT-2h já processado
BTCUSDT-4h já processado
BTCUSDT-8h já processado
BTCUSDT-1d já processado


In [9]:
def une_timeframes(timeframes=timeframes):
    # junta todos os timeframes em um df só

    # pega o primeiro timeframe com as colunas desejadas
    df = pd.read_csv(f"./Dados/Processados/BTCUSDT-{timeframes[0]}.csv", index_col = "Close time"
                     , usecols=["Close time","Open", "High", "Low", "Close", "Volume", "Number of trades"])
    #df.index = pd.to_datetime(df.index, unit="ms")

    # coloca o sufixo nas colunas do menor timeframe
    df.columns = df.columns+f"_{timeframes[0]}"
    print(timeframes[0])

    # junta todos os outros timeframes e coloca o sufixo correto
    for timeframe in timeframes[1:]:
        print(timeframe)
        df2 = pd.read_csv(f"./Dados/Processados/BTCUSDT-{timeframe}.csv", index_col = "Close time",
                         usecols=["Close time","Open", "High", "Low", "Close", "Volume", "Number of trades"])
        df2.columns = df2.columns+f"_{timeframe}"
        df = df.merge(df2, how="left", on=["Close time"])#, suffixes=(None, f"_{timeframe}"))


    # transforma o index em datetime
    df.index = pd.to_datetime(df.index, unit="ms")

    #inverte a ordem do df pq vamos usar ele do fim pro começo pra fazer as janelas
    df.sort_index(axis=0, inplace = True, ascending=False)
    return(df)
df = une_timeframes(timeframes)
df

1m
5m
15m
30m
1h
2h
4h
8h
1d


,Open_1m,High_1m,Low_1m,Close_1m,Volume_1m,Number of trades_1m,Open_5m,High_5m,Low_5m,Close_5m,...,Low_8h,Close_8h,Volume_8h,Number of trades_8h,Open_1d,High_1d,Low_1d,Close_1d,Volume_1d,Number of trades_1d
Close time,,,,,,,,,,,,,,,,,,,,,
2022-11-30 23:59:59.999,17164.11,17167.99,17160.00,17163.64,143.984230,3565,17175.50,17215.59,17150.70,17163.64,...,16705.0,17163.64,115168.64732,2458897.0,16442.91,17249.0,16428.3,17163.64,303019.80719,6519330.0
2022-11-30 23:58:59.999,17161.26,17170.27,17159.51,17164.87,129.203750,3760,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-30 23:57:59.999,17165.50,17170.80,17156.77,17161.26,160.506230,4056,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-30 23:56:59.999,17173.33,17175.40,17150.70,17167.16,322.428780,7797,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-30 23:55:59.999,17175.50,17215.59,17170.01,17173.33,531.365170,6445,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-17 04:04:59.999,4261.48,4261.48,4261.48,4261.48,0.140796,1,4261.48,4280.56,4261.48,4261.48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-17 04:03:59.999,4261.48,4261.48,4261.48,4261.48,0.012008,3,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-17 04:02:59.999,4280.56,4280.56,4280.56,4280.56,0.261074,2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Problemas:
- Se for usar GAF image vai ter uma imagem 20x20 por coluna (supondo que estamos olhando 20 períodos no passado por vez)
- Não está claro se 1D CNN é uma boa alternativa para timeseries.

# Soluções:
#### 1
- Usar GAF de qualquer maneira para tentar replicar os resultados do trabalho do BARRA, usando apenas os preços de fechamento de 4 timeframes e ver se os resultados melhoram com CSVM
- Depois se eu quiser adiciono mais dados, mas a imagem vai ficar bem grande. (5 colunas de t-20 resulta em uma imagem de 100x100). GAF usando mais dados seria o melhor dos mundos.

#### 2
- Usar 1D CNN com todos os dados disponíveis
- Metodo completamente diferente do do BARRA. Não usa GAF images. Provavelmente mais fácil

### A partir daqui são só testes

In [10]:
# cria janela de precos
lookback = 20
linha = 0
# tentar fazer um loop pra pegar tudo
df_janela = pd.DataFrame()
#for linha in range(len(df)):
for timeframe in timeframes:
    # insere na ultima coluna; com o nome de Close_timeframe ;
    # df a partir da linha atual até linha atual + lookback
    df_janela.insert(len(df_janela.columns),(f"Close_{timeframe}"),df[f"Close_{timeframe}"].iloc[linha:].dropna()[:linha+lookback].to_list())
df_janela

,Close_1m,Close_5m,Close_15m,Close_30m,Close_1h,Close_2h,Close_4h,Close_8h,Close_1d
0,17163.64,17163.64,17163.64,17163.64,17163.64,17163.64,17163.64,17163.64,17163.64
1,17164.87,17174.44,17222.12,17177.06,17148.29,17106.65,17062.85,16865.64,16442.53
2,17161.26,17203.21,17177.06,17148.29,17106.65,17062.85,16865.64,16884.18,16212.91
3,17167.16,17222.12,17169.93,17107.43,17097.19,16789.66,16879.32,16442.53,16428.78
4,17173.33,17183.44,17148.29,17106.65,17062.85,16865.64,16884.18,16393.48,16458.57
5,17174.44,17145.74,17087.65,17041.71,16921.44,16810.12,16845.25,16463.31,16522.14
6,17197.18,17177.06,17107.43,17097.19,16789.66,16879.32,16442.53,16212.91,16598.95
7,17191.71,17163.30,17095.53,17064.40,16861.98,16879.49,16426.56,16146.26,16603.11
8,17201.51,17163.01,17106.65,17062.85,16865.64,16884.18,16393.48,16222.06,16226.94
9,17185.69,17169.93,17064.97,17115.80,16847.01,16864.10,16497.64,16428.78,15781.29


### Falhas